<a href="https://colab.research.google.com/github/R4HUL-ROY/Machine-Learning/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment on Naive Bayes Classification**

**NAME : RAHUL ROY**

**MCA 2nd Sem**

**ROLL - C91/MCA/202032**

In [ ]:
import numpy as np
import pandas as pd

**CHOOSE AND READ DATASET**

In [ ]:
## DATASET - 1
## play_golf dataset

data_dict = {
    "outlook" : ['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy', 'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast', 'overcast', 'rainy'],
    "temperature" : ['hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool', 'mild', 'cool', 'mild', 'mild', 'mild', 'hot', 'mild'],
    "humidity" : ['high', 'high', 'high', 'high', 'normal', 'normal', 'normal', 'high', 'normal', 'normal', 'normal', 'high', 'normal', 'high'],
    "windy" : ['false', 'true', 'false', 'false', 'false', 'true', 'true', 'false', 'false', 'false', 'true', 'true', 'false', 'true']
}

result_dict = {
    "play" : ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']
}

data_df = pd.DataFrame(data_dict)
result_df = pd.DataFrame(result_dict)

x = np.array(data_df)
y = np.array(result_df)


In [ ]:
## DATASET - 2
# CAR DATASET    link : http://archive.ics.uci.edu/ml/datasets/Car+Evaluation


df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/car.csv')

## renaming the column names
col_name = ['buying', 'maint', 'doors' , 'persons', 'lug_boot', 'safety', 'class']
df.columns = col_name

data_df = df[['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']]
result_df = df['class']

x = np.array(data_df)
y = np.array(result_df)

print(df.shape)
print(x.shape)
print(y.shape)

(1727, 7)
(1727, 6)
(1727,)


In [ ]:
# DATASET - 3
# Link of this datset : http://archive.ics.uci.edu/ml/datasets/Tic-Tac-Toe+Endgame

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/tic_tac_toe.csv')
df.columns

data_df = df[['x', 'x.1', 'x.2', 'x.3', 'o', 'o.1', 'x.4', 'o.2', 'o.3']]
result_df = df['positive']

# print(data_df)
# print(result_df)

x = np.array(data_df)
y = np.array(result_df)

In [ ]:
# DATASET 4

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/nursery.csv')

col_name = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'class']
df.columns = col_name

data_df = df[['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']]
result_df = df['class']

x = np.array(data_df)
y = np.array(result_df)

# print(entire_df.shape)
print(x.shape)
print(y.shape)

(12959, 8)
(12959,)


In [ ]:
## splitting the dataset into x_train, y_train , x_test and y_test

def split_data(x , y):
  import random

  row, col = x.shape
  row_count_for_train = round(row*0.8)   # 80% selected as train data

  train_rows = []   #containing the indexes of rows considered for training
  test_rows = []    #containing the indexes of rows considered for testing 

  while len(train_rows) < row_count_for_train:
    random_num = random.randint(0, row-1)  # generating random number
    if random_num not in train_rows:
      train_rows.append(random_num)

  for i in range(row):
    if i not in train_rows:
      test_rows.append(i)
  random.shuffle(test_rows)  #shuffling because our test_rows was containing value in sorted order    

  x_train = x[train_rows,]
  y_train = y[train_rows,]

  x_test = x[test_rows,]
  y_test = y[test_rows,]

  return x_train, y_train, x_test, y_test
  

In [ ]:


##helper function for posterior probability
def prob(x_train, y_train, col_index , feature , ci):  ## feature is the catagory for that feature ## ci is label
  count =0
  for i in range(x_train.shape[0]):
    if x_train[i][col_index] == feature and y_train[i] == ci:   ## if train row match with catagory of test row and outcome also same as asking label,  increase count by 1
      count+=1
  return count    ## returns number of rows that has same catagory and same outcome


#calculate posterior probability for all ci for a single test_row and return posterior probability dictionary {label-1 : posterior probability , label-2 : ..... }
def calculate_likelyhood(x_train, y_train, frame , this_test_row):  # likelyhood is the p(x|H) part   (posterior_prob = likelyhood * prior_prob / evidence )
  likelyhood_dict = {}
  for ci, val in frame.items():
    res = 1
    for i in range(len(this_test_row)) :
      res = res* ( prob(x_train , y_train,  i, this_test_row[i], ci)/val )   ## multiplying individual probability  PI(k=1 upto n) p(Xk | ci) X is test row, ci is label
    likelyhood_dict[ci] = res
  return likelyhood_dict    ## returning posterior probability dictionary for where key is a class label and value is the corresponding posterior probability


def find_posterior_prob_each_label(frame,prior_dict, x_train, y_train, this_test_row):
  likelyhood_dict = calculate_likelyhood(x_train, y_train, frame, this_test_row)  
  final_pred_for_each_label = {}
  for ci, val in frame.items(): # iterate over each label
    final_pred_for_each_label[ci] = likelyhood_dict[ci] * prior_dict[ci]    ## posterior_prob = likelyhood * prior_prob   P(Ci | X) = P(X | Ci) * p(Ci)
  return final_pred_for_each_label  ## returns final prediction for each label, from which the label with maximum probability is our classification label for this test row


'''helper method for fit_model'''
#calculate prior probability (called inside fit_model method)
def calculate_prior_prob(frame , row_for_train):
  prior_prob_dict = {}
  for ci in frame.keys():
    prior_prob_dict[ci] = frame[ci]/row_for_train     # calculate no_of_row_with_a_label_present (label is "yes"/"no"/..) / total_no_of_row_in_train_dataset 
  return prior_prob_dict  ## contains all the prior probability for each label

## fit the model (called inside fit_and_predict function)
def fit_model(x_train, y_train):
  row_for_train = len(x_train)   #80% of the original dataset
  label , freq = np.unique(y_train, return_counts = True)
  frame = dict(zip(label,freq))    ## zip all the label along with thier frequency on x_test data
  prior_dict = calculate_prior_prob(frame , row_for_train)
  return frame , prior_dict    ## frame contains label and its frequency in train dataset


def fit_and_predict(x_train, y_train , x_test):
  frame, prior_dict =  fit_model(x_train, y_train)   ## fitting the data into model by calling fit function
  y_pred = []
  for this_test_row in x_test:  #iterate over each test data sample
    posterior_pred_dict = find_posterior_prob_each_label(frame, prior_dict, x_train, y_train, this_test_row)## this returns posterior probability for each label, for a particular test data sample
    # print(posterior_pred_dict)
    key_with_max_prob = max(posterior_pred_dict, key=posterior_pred_dict.get) ## this find the label with max posterior probability among all the labels probability
    y_pred.append(key_with_max_prob)
  return y_pred   # returns the predicted label list, for each test data sample


**Main Module for function calling**

In [ ]:
""" main module calling split_data & fit_and_predict function """

## splitting data
x_train, y_train , x_test, y_test = split_data(x , y)

## Fit and predicting the result fot test dataset
y_predicted = fit_and_predict(x_train, y_train, x_test)  ## make final prediction 

print(y_predicted)    

['priority', 'priority', 'priority', 'not_recom', 'spec_prior', 'spec_prior', 'not_recom', 'spec_prior', 'not_recom', 'not_recom', 'spec_prior', 'priority', 'not_recom', 'spec_prior', 'not_recom', 'priority', 'not_recom', 'priority', 'spec_prior', 'not_recom', 'priority', 'spec_prior', 'spec_prior', 'not_recom', 'spec_prior', 'spec_prior', 'not_recom', 'spec_prior', 'not_recom', 'priority', 'not_recom', 'not_recom', 'spec_prior', 'spec_prior', 'priority', 'priority', 'priority', 'priority', 'priority', 'not_recom', 'spec_prior', 'priority', 'priority', 'priority', 'spec_prior', 'spec_prior', 'not_recom', 'priority', 'spec_prior', 'priority', 'priority', 'not_recom', 'not_recom', 'not_recom', 'not_recom', 'priority', 'not_recom', 'priority', 'priority', 'not_recom', 'not_recom', 'not_recom', 'priority', 'priority', 'not_recom', 'not_recom', 'spec_prior', 'priority', 'spec_prior', 'spec_prior', 'spec_prior', 'not_recom', 'not_recom', 'spec_prior', 'priority', 'not_recom', 'not_recom', 'n

In [ ]:
## showing results

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

matrix = confusion_matrix(y_test, y_predicted)

print(f"\nconfusuin matrix :\n {matrix}\n\n")


diagonal_sum = 0
total_sum = 0

  #calculating diagonal sum and total sum from confusion matrix
for i in range(matrix.shape[0]):
  for j in range(matrix.shape[1]):
    if i == j:
      diagonal_sum += matrix[i,j]
    total_sum += matrix[i,j]
print(f"\nAccuracy = {(diagonal_sum/total_sum)*100}%\n\n")


print(f"{classification_report(y_test, y_predicted)}" )


confusuin matrix :
 [[896   0   0   0]
 [  0 762  76   0]
 [  0  84 714   0]
 [  0  57   0   3]]



Accuracy = 91.62808641975309%


              precision    recall  f1-score   support

   not_recom       1.00      1.00      1.00       896
    priority       0.84      0.91      0.88       838
  spec_prior       0.90      0.89      0.90       798
  very_recom       1.00      0.05      0.10        60

    accuracy                           0.92      2592
   macro avg       0.94      0.71      0.72      2592
weighted avg       0.92      0.92      0.91      2592

